In [ ]:
import pandas as pd
from datetime import datetime
pd.options.plotting.backend = "plotly"

services_baristas = pd.read_csv("../calendar/baristas.csv", sep=";",decimal=',',low_memory=False, index_col=False)
services_baristas['mois'] = pd.to_datetime(services_baristas['mois'])
services_baristas.head()

In [ ]:
creneaux_annules = services_baristas[services_baristas['role'] == "CRENEAU ANNULE"][['mois', 'role']].groupby('mois').count()
creneaux_annules['year'] = creneaux_annules.index.year
creneaux_annules = creneaux_annules.reset_index().set_index(['mois', 'year']).unstack('year').fillna(0)
creneaux_annules.columns = creneaux_annules.columns.droplevel(0)
creneaux_annules.plot(kind='bar')

In [ ]:
nb_baristas = services_baristas[services_baristas['role'] != "CRENEAU ANNULE"].groupby(['mois','nom','telephone']).count().rename(columns= {'role': 'nb'})
nb_baristas = nb_baristas.groupby('mois').aggregate(func=['count', 'sum'])
nb_baristas.columns = nb_baristas.columns.droplevel(0)
nb_baristas = nb_baristas.rename(columns={'count': 'nb baristas', 'sum': 'nb services réalisés'})
nb_baristas['year'] = nb_baristas.index.year
nb_baristas = nb_baristas.reset_index().set_index(['mois', 'year']).unstack('year').fillna(0)

In [ ]:
nb_baristas[['nb baristas']].stack('year').groupby('year').sum().plot(kind='bar')

In [ ]:
nb_baristas[['nb services réalisés']].stack('year').groupby('year').sum().plot(kind='bar')

In [ ]:
nb_baristas['nb baristas'].plot(kind='bar')

In [ ]:
nb_baristas['nb services réalisés'].plot(kind='bar')

In [ ]:
services = services_baristas[services_baristas['role'] == "REFERENT"].groupby(['mois','nom','telephone']).count().rename(columns= {'role': 'nb'})
services = services.groupby('mois').aggregate(func=['count', 'sum'])
services.columns = services.columns.droplevel(0)
services['year'] = services.index.year

services = services.reset_index().set_index(['mois', 'year']).unstack('year').fillna(0)

In [ ]:
services['count'].plot(kind='bar')

In [ ]:
services['sum'].plot(kind='bar')

In [ ]:
limit_date = datetime.now().replace(year=datetime.now().year - 1)
baristas_t2n = services_baristas.groupby(['telephone'])['nom','role'].first()
baristas = services_baristas[services_baristas['mois'] > limit_date][services_baristas['role'] != "CRENEAU ANNULE"].groupby(['telephone'])[['role']].count()
baristas['nb'] = services_baristas[services_baristas['mois'] > limit_date][services_baristas['role'] != "CRENEAU ANNULE"].groupby(['telephone'])[['role']].count()
baristas.sort_values(by='nb', ascending=False, inplace=True)
baristas['nom'] = baristas.index.map(baristas_t2n['nom'])
baristas = baristas.reset_index()[['nom', 'nb']].groupby('nom')[['nb']].sum().sort_values(by='nb', ascending=False)
baristas.plot()

In [ ]:
baristas.head(30)

In [ ]:
len(baristas[baristas[['nb']] < 2].dropna().reset_index()['nom'].unique())

In [ ]:
len(baristas[baristas[['nb']] > 20].dropna().reset_index()['nom'].unique())

In [ ]:
len(baristas[baristas[['nb']] > 40].dropna().reset_index()['nom'].unique())

In [ ]:
len(baristas[baristas[['nb']] > 60].dropna().reset_index()['nom'].unique())

In [ ]:
len(baristas.dropna().reset_index()['nom'].unique())

In [ ]:
barista_std = baristas.reset_index()
barista_std[barista_std.index == len(barista_std)/2].reset_index()['nb'][0]

In [ ]:
baristas['nb'].mean()